In [19]:
from __future__ import division      #除数可以显示为float

from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd

import time                 #使用time stamp
import datetime             

import matplotlib.pyplot as plt

import math


# 读取归一化后的数据
Data_Csv = pd.DataFrame()
Data_Csv = pd.read_csv("Data_Washed.csv")

Data_Csv = Data_Csv.fillna(0)

# 生成测试数据
# X =  np.array(Data_Csv.loc[:,['MACD_macd', 'EMA_gap', 'KDJ_J', 'RSI', 'MOM', 'MOM_gap', 'OBV', \
#         'Long_Short_Rate_OBV', 'KDJ_J_pre', 'MOM_pre', 'MOM_gap_pre', \
#        'OBV_pre', 'Long_Short_Rate_OBV_pre', 'MACD_macd_pre','EMA_gap_pre']])

# 生成测试数据
X =  np.array(Data_Csv.iloc[:,4:])
# 生成测试结果
y = np.array(Data_Csv.loc[:,"win_rate"])

print X
print y

[[  1.06877860e-02   2.60117800e-03   8.08660800e-03 ...,   8.61921557e+09
   -2.65632408e+08   3.54176544e+08]
 [ -1.13615270e-02  -2.05305700e-03  -9.30847100e-03 ...,   8.28425945e+09
   -2.34469286e+08   3.34956123e+08]
 [ -2.42150000e-02  -9.44037100e-03  -1.47746290e-02 ...,   7.86560480e+09
    2.22688645e+08   4.18654653e+08]
 ..., 
 [ -5.03564200e-03  -5.74005740e-02   5.23649310e-02 ...,   5.02408311e+10
    1.24747496e+08   3.38600346e+08]
 [ -3.51478700e-03  -3.94386450e-02   3.59238580e-02 ...,   5.00333585e+10
   -1.38315111e+08   2.07472666e+08]
 [ -1.00479700e-02  -2.96417530e-02   1.95937830e-02 ...,   4.98617024e+10
   -1.02993661e+08   1.71656102e+08]]
[-1 -1 -1 ..., -1 -1  1]


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
# 模型训练
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print('train score: {train_score:.6f}; test score: {test_score:.6f}'.format(
    train_score=train_score, test_score=test_score))

train score: 0.764896; test score: 0.787823


In [22]:
np.set_printoptions(threshold='nan') 
# print y_test
# print np.array(model.predict(X_test))

In [26]:
# 样本预测
y_pred = model.predict(X_test)

test_t = list(np.where(np.array(y_test) == -1))
pre_t = list(np.where(np.array(y_pred) == 1))

print "实际为-1，但是预测为1"
print len(set(test_t[0])&set(pre_t[0]))



test_t = list(np.where(np.array(y_test) == 1))
pre_t = list(np.where(np.array(y_pred) == -1))

print "实际为1，但是预测为-1"
print len(set(test_t[0])&set(pre_t[0]))



test_t = list(np.where(np.array(y_test) == 1))
pre_t = list(np.where(np.array(y_pred) == 1))

print "实际为1，但是预测为1"
print len(set(test_t[0])&set(pre_t[0]))

# print y_pred[np.array(y_pred) == 1]
# print('matchs: {0}/{1}'.format(np.equal(y_pred, y_test).shape[0], y_test.shape[0]))

实际为-1，但是预测为1
12
实际为1，但是预测为-1
96
实际为1，但是预测为1
11


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision_score(y_test,y_pred , average='binary')

0.47826086956521741

In [28]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

# 增加多项式预处理
def polynomial_model(degree=1, **kwarg):
    polynomial_features = PolynomialFeatures(degree=degree,
                                             include_bias=False)
    logistic_regression = LogisticRegression(**kwarg)
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("logistic_regression", logistic_regression)])
    return pipeline

model = polynomial_model(degree=2, penalty='l1')

start = time.clock()
model.fit(X_train, y_train)

train_score = model.score(X_train, y_train)
cv_score = model.score(X_test, y_test)
print('elaspe: {0:.6f}; train_score: {1:0.6f}; cv_score: {2:.6f}'.format(
    time.clock()-start, train_score, cv_score))



elaspe: 64.663294; train_score: 0.781062; cv_score: 0.800738


In [29]:
print precision_score(y_test,y_pred , average='binary')

0.478260869565


In [30]:
#  pipeline.named_steps给出了流水线上每一步使用的学习器
#  coef_里面保存的就是模型参数
logistic_regression = model.named_steps['logistic_regression']
print('model parameters shape: {0}; count of non-zero element: {1}'.format(
    logistic_regression.coef_.shape, 
    np.count_nonzero(logistic_regression.coef_)))

model parameters shape: (1L, 152L); count of non-zero element: 76
